In [13]:
# made by Gibeom LEE, HI LAB
# csv파일 여러개를 일정한 간격을 두어 합쳐줍니다. (다른 열)

In [14]:
import os
import pandas as pd
import numpy as np
import glob

In [15]:
# 설정 ----------------------------
folder = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250904_다 찍기\csv파일\전압 전류 전하\전류"

file_name_str = "Pristine_4cm_1.83Hz Coated_4cm_1.74Hz Pristine_4cm_2.44Hz Coated_4cm_2.65Hz Pristine_4cm_Many Coated_4cm_3.98Hz"  # 폴더 내에 일부 파일만 변경하려면 파일명 기입 필요

gap_ratio = 0.2  # 각 파일 사이의 시간 간격 비율 ex) 20%


In [16]:
# 폴더 이름 추출
folder_name = os.path.basename(folder)
output_filename = f"{folder_name}_merge.csv"
output_path = os.path.join(folder, output_filename)

# 병합 함수 -----------------------
def merge_data(file_name_str, folder, output_path, gap_ratio):
    file_names = file_name_str.split()
    
    # 폴더 내 일부 csv가져오기
    #file_paths = [os.path.join(folder, f"{name}.csv") for name in file_names]
    # 폴더 내 모든 csv가져오기 %%%%%%%%%%%%%%%%%%%%%% 00Merge 파일 없애고 실행시킬것!!! %%%%%%%%%%%%%%%%%
    file_paths = glob.glob(os.path.join(folder, "*.csv"))
    
    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]
    shifted_data = []  # 이동된 시간축 데이터 저장용
    current_time_offset = 0.0  # 누적 시간 offset

    for i, df in enumerate(dataframes):
        time_vals = df.iloc[:, 0].values
        voltage_vals = df.iloc[:, 1].values

        # 현재 파일 시간 누적 shift
        shifted_time = time_vals + current_time_offset

        # 데이터프레임으로 저장
        shifted_df = pd.DataFrame({
            "Time": shifted_time,
            f"V{i+1}": voltage_vals
        })
        shifted_data.append(shifted_df)

        # 시간 이동 계산을 위한 값 추출
        last_time = time_vals[-1]
        rounded_duration = round(last_time)  # 이 파일의 총 시간(정수 기준)
        time_gap = rounded_duration * gap_ratio

        # 다음 파일을 위한 누적 offset 업데이트
        current_time_offset += rounded_duration + time_gap

    # 병합: 시간 기준 outer join
    merged_df = shifted_data[0]
    for df in shifted_data[1:]:
        merged_df = pd.merge(merged_df, df, on="Time", how="outer")

    # 정렬 및 저장
    merged_df = merged_df.sort_values("Time").reset_index(drop=True)
    merged_df.to_csv(output_path, index=False, header=False, float_format="%.6f")

    print(f"✅ 시간축 간격을 적용하여 병합된 파일이 저장되었습니다: {output_path}")


# 실행 ----------------------------
merge_data(file_name_str, folder, output_path, gap_ratio)

✅ 시간축 간격을 적용하여 병합된 파일이 저장되었습니다: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250904_다 찍기\csv파일\전압 전류 전하\전류\전류_merge.csv
